In [ ]:
from experiments.attn.victory import *

In [ ]:
results = {}
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [8, 16, 32, 64, 128]:
            print(boardsize, depth, width)
            results[boardsize, depth, width] = run_trial(boardsize=boardsize, n_layers=depth, D=width)

In [ ]:
from pathlib import Path
import pickle
Path('output/experiments-victory.pkl').write_bytes(pickle.dumps(results))

In [ ]:
pickle.loads(Path('output/experiments-victory.pkl').read_bytes())

In [ ]:
from pavlov import stats, runs, storage
Model = common.FCModel
run = runs.new_run('experiments-victory', model=Model.__name__)

In [ ]:
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [8, 16, 32, 64, 128]:
            storage.snapshot(run, {'losses': results[boardsize, depth, width].to_dict()}, boardsize=boardsize, depth=depth, width=width)

In [ ]:
from pavlov import stats, runs, storage, files

In [ ]:
snapshots = pd.DataFrame.from_dict(storage.snapshots(run), orient='index')
df = {}
for i, row in snapshots.iterrows():
    df[row.boardsize, row.depth, row.width] = storage.load_snapshot(run, i)

In [ ]:
df